In [2]:
import os
import sys
from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define your Project Root
project_root_path = '/content/drive/MyDrive/AEE Setup FT'

# 3. Change the working directory to the root
os.chdir(project_root_path)

# 4. Add the root to Python's search path so it finds 'src'
if project_root_path not in sys.path:
    sys.path.append(project_root_path)

# 5. Verify it works
print(f"Current working directory: {os.getcwd()}")
print("Setup complete. You can now import from 'src'!")

Mounted at /content/drive
Current working directory: /content/drive/MyDrive/AEE Setup FT
Setup complete. You can now import from 'src'!


In [3]:
import sys
import torch

print("="*60)
print("ENVIRONMENT CHECK")
print("="*60)
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Device capability: {torch.cuda.get_device_capability(0)}")
    print(f"Total VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ WARNING: CUDA not available. Training will be VERY slow on CPU.")

print("="*60)

ENVIRONMENT CHECK
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch version: 2.9.0+cu128
CUDA available: True
CUDA version: 12.8
Device name: Tesla T4
Device capability: (7, 5)
Total VRAM: 14.56 GB


In [4]:
!pip -q install -U bitsandbytes>=0.46.1 accelerate transformers peft

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# Step 1: Base model + adapter
base_model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit"
adapter_path = "./checkpoint-100"

# 4-bit config (GPU)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)

# Step 2: Load LoRA adapters
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

# (optional) slightly faster inference
torch.set_float32_matmul_precision("high")

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:246: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [6]:
import time
from typing import Dict, Any
import torch

def query_ft(question: str, max_new_tokens: int = 96) -> Dict[str, Any]:
    t0 = time.time()

    prompt = (
        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
        "You are a financial analyst for Uber.\n"
        "<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
        f"{question}\n"
        "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,          # greedy = faster + stable
            temperature=None,         # not used when do_sample=False
            top_p=None,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # safer extraction: take everything after the user question
    # (split on the last occurrence of question)
    idx = text.rfind(question)
    answer = text[idx + len(question):].strip() if idx != -1 else text.strip()

    latency = int((time.time() - t0) * 1000)
    return {"answer": answer, "latency": latency}

In [7]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5cedf608c4708ca41f2569d8b710af497f6959bfd6cc8c9061c276fd15b529bf
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [8]:
from rouge_score import rouge_scorer

rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def calculate_rouge(pred: str, ref: str) -> float:
    scores = rouge.score(ref, pred)
    return scores['rougeL'].fmeasure

def llm_judge(question: str, ground_truth: str, prediction: str) -> int:
    if not prediction or not prediction.strip():
        return 1

    prompt = f"""You are an impartial judge evaluating the quality of an AI-generated answer.

    Question: {question}
    Ground Truth: {ground_truth}
    Prediction: {prediction}

    Rate the Prediction on a scale of 1 to 5 based on Faithfulness and Accuracy compared to the Ground Truth.
    1 = Completely wrong or irrelevant
    3 = Partially correct but missing details
    5 = Perfect match in information and intent

    Return ONLY the integer score (1-5)."""

    try:
        resp = oa.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        score = int(resp.choices[0].message.content.strip())
        return max(1, min(5, score))
    except:
        return 1

In [9]:
from typing import List, Dict
import json

def load_jsonl(filepath: str) -> List[Dict]:
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

test_file = "golden_test_set.jsonl"
test_set = load_jsonl(test_file)

print(f"✅ Loaded {len(test_set)} test cases from {test_file}")
if test_set:
    print(f"Sample: {test_set[0]['question']}")

✅ Loaded 240 test cases from golden_test_set.jsonl
Sample: What can be inferred about the future relationship between Uber and its Drivers based on the current challenges mentioned?


In [10]:
import pandas as pd
from tqdm import tqdm

results = []
limit = 50

ENABLE_JUDGE = True

print(f"🚀 Starting evaluation on {min(len(test_set), limit)} samples...")

for i, item in enumerate(tqdm(test_set[:limit], desc="Eval")):
    q = item["question"]
    gt = item["answer"]
    category = item.get("category", "Unknown")

    row = {"question": q, "category": category, "original_answer": gt}

    # --- FT ---
    try:
        ft_res = query_ft(q, max_new_tokens=96)
        ft_ans = ft_res["answer"]
        ft_lat = ft_res["latency"]
        ft_rouge = calculate_rouge(ft_ans, gt)
        ft_score = llm_judge(q, gt, ft_ans) if ENABLE_JUDGE else None
    except Exception as e:
        ft_ans, ft_lat, ft_rouge, ft_score = "ERROR", None, None, None

    row.update({
        "FT_Answer": ft_ans,
        "FT_Latency": ft_lat,
        "FT_ROUGE": ft_rouge,
        "FT_Score": ft_score
    })

    results.append(row)

    # Save partial every 5 rows (so you never lose work)
    if (i + 1) % 5 == 0:
        pd.DataFrame(results).to_csv("eval_ft_partial.csv", index=False)

df = pd.DataFrame(results)
df.to_csv("eval_ft_results.csv", index=False)
print("✅ Evaluation Complete! Saved: eval_ft_results.csv")


🚀 Starting evaluation on 50 samples...


Eval: 100%|██████████| 50/50 [10:28<00:00, 12.58s/it]

✅ Evaluation Complete! Saved: eval_ft_results.csv
